In [56]:
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity


In [57]:
%%time
query_list = []
doc_list = []

with open('./query_list.txt',"r") as querys:
    for query in querys:
        query_list.append(query.strip())
with open('./doc_list.txt',"r") as docs:
    for doc in docs:
        doc_list.append(doc.strip())

Wall time: 4.99 ms


In [58]:
                                        
query_word = []                                #製造要變成 tfidf 的原始資料矩陣
for query in query_list:
    with open('./Query/'+query,"r") as f:
        a=' '
        for line in f:
            s=line.strip()[:-2]
            a+=s
        query_word.append(a)

doc_word = []
for doc in doc_list:
    with open('./Document/'+doc,"r") as f:
        a = ' '
        f = f.readlines()[3:]
        lines = [line.strip()[:-2] for line in f]
        b = ''.join(lines)
        doc_word.append(b)

In [59]:
%%time
tfidf_model = TfidfVectorizer(token_pattern='[0-9]+')                #用套件變成tfidf矩陣
doc_vector = tfidf_model.fit_transform(doc_word)
q_vector = tfidf_model.transform(query_word)
q_vector = q_vector.toarray()
q_len , word_len = q_vector.shape

print(doc_vector.shape)
print(q_vector.shape)

(2265, 13290)
(800, 13290)
Wall time: 581 ms


In [60]:
%%time

rank = cosine_similarity(q_vector,doc_vector)    #算cosine 值
def roc(q_vector,rank,doc_vector):               #做出訓練的演算法函數
    for i in range(q_len):
        a = np.argsort(rank[i])
        a1 = a[:5]
        a2 = list(reversed(a))[:5]
        sum1 = []
        sum2 = []
        for j in a1:
            if sum1==[]:
                sum1=doc_vector[j]
            else:
                sum1+=doc_vector[j]
        for k in a2:
            if sum2==[]:
                sum2=doc_vector[k]
            else:
                sum2+=doc_vector[k]
        q_vector[i] = alpha*q_vector[i]+beta*(1/len(a2))*sum2-gama*(1/len(a1))*sum1
    rank = cosine_similarity(q_vector,doc_vector)
    return q_vector ,rank


Wall time: 321 ms


In [61]:
%%time
alpha=0.7
beta=0.4
gama=0.3
for i in range(5):                  #演算法函數跑五次
    q_vector,rank=roc(q_vector,rank,doc_vector)
new_rank = cosine_similarity(q_vector,doc_vector)
print(new_rank.shape)

(800, 2265)
Wall time: 13.3 s


In [62]:
f=open("submission.txt","w")                 #選出50筆由大到小寫入
f.write("Query,RetrievedDocuments\n")
i=0
for query in query_list:
    f.write(query+",")
    b=np.argsort(new_rank[i])
    b1=list(reversed(b))[:50]
    for t in b1:
        f.write(doc_list[t]+" ")
    f.write("\n")
    i+=1
f.close()                                #到此結束